### Анализ базы данных с целью формулирования ценностного предложения для нового продукта

### Описание данных
В БД содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![альт](https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&width=2730&userId=95479034-1cd7-473a-8bed-350b991312c6&cache=v2 "описание при наведении")

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'user', # имя пользователя
 'pwd': 'pwd', # пароль
 'host': 'host',
 'port': port , # порт подключения
 'db': 'db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
#Функция для выводы SQL запроса
def query(query):
    return pd.io.sql.read_sql(query, con = engine) 

In [4]:
books = query('''SELECT * FROM books''')

authors = query('''SELECT * FROM authors''')

publishers = query('''SELECT * FROM publishers''')

ratings = query('''SELECT * FROM ratings''')

reviews = query('''SELECT * FROM reviews''')

In [5]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Количество книг вышедшее после 1 января 2000г.

In [10]:
#Количество книг вышедшее после 1 января 2000 года
query('''SELECT DISTINCT COUNT(title) AS количество_книг 
         FROM books WHERE publication_date > ('2000-01-01')''')

,количество_книг
0,819


В БД содержится 819 книг, которые вышли после 1.01.2000 года

In [11]:
#Количество обзоров и средняя оценка для каждой книги
title_rev_rating = query('''SELECT title AS название_книги, кол_обзоров, средняя_оценка FROM books 
JOIN 
(SELECT book_id, COUNT(review_id) AS кол_обзоров FROM reviews GROUP BY book_id) AS count_rew
ON count_rew.book_id = books.book_id 
JOIN
(SELECT book_id, AVG(rating) AS средняя_оценка FROM ratings GROUP BY book_id) AS avg_rating
ON books.book_id = avg_rating.book_id
''')

In [12]:
title_rev_rating.head()

,название_книги,кол_обзоров,средняя_оценка
0,The Body in the Library (Miss Marple #3),2,4.500000
1,Galápagos,2,4.500000
2,A Tree Grows in Brooklyn,5,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2,4.000000
4,The Prophet,4,4.285714


На выходе таблица, где для каждой книги мы имеем количество обзоров и среднюю оценку этой книги

In [13]:
#Издательство с наибольшим числом выпущенных книг, где страниц больше 50
query('''SELECT publisher FROM publishers 
         WHERE publisher_id IN 
         (SELECT publisher_id FROM 
         (SELECT publisher_id, COUNT(title) AS количество_книг FROM books 
         WHERE num_pages > 50 
         GROUP BY publisher_id 
         ORDER BY  количество_книг DESC
         LIMIT 1) AS max_count)

''')

,publisher
0,Penguin Books


Издательство с наибольшим числом выпущенных книг - Penguin Books

In [14]:
#Автор с самой высокой средней оценкой книг, где количество оценок книг от 50
query(''' SELECT author FROM authors WHERE author_id IN 
        (SELECT author_id FROM books 
         WHERE book_id IN 
        (SELECT book_id FROM 
        (SELECT book_id, AVG(rating) AS средняя_оценка, COUNT(rating_id) FROM ratings 
         GROUP BY book_id HAVING COUNT(rating_id) > 50 
         ORDER BY средняя_оценка DESC LIMIT 1) AS book_id_max_avg_rat))''')

,author
0,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (Привет, Гарри Поттер :)) 

In [15]:
#Среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query('''
SELECT AVG(count_rew.count) AS среднее_кол_обзоров FROM 
(SELECT COUNT(review_id) FROM reviews
WHERE username IN
(SELECT username FROM 
(SELECT username, COUNT(rating_id) FROM ratings 
GROUP BY username
HAVING COUNT(rating_id) > 50) AS us_n)
GROUP BY username) as count_rew''')

,среднее_кол_обзоров
0,24.333333


24,3 среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Вывод: Действия, которые мы проделали, помогут нам определить наиболее популярные книги и авторов, что бы предложить нашему пользователю. Если мы угодим ему с подбором книги, то скорее всего, он будет делать покупки повторно. 